# Taichung City Bus Timer

#### 台中市 公車到站時刻查詢
http://citybus.taichung.gov.tw/ibus/RealRoute.aspx  

#### 站牌名稱 編號 座標
http://citybus.taichung.gov.tw/ibus/RealRoute/aspx/RealRoute.ashx?Lang=Cht&BusType=0&Data=53_,1&Type=GetStop

#### 站牌編號 到站時刻 
http://citybus.taichung.gov.tw/ibus/RealRoute/aspx/RealRoute.ashx?Lang=Cht&BusType=0&Data=53_,1&Type=GetFreshData

In [ ]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.path.pardir, os.path.sep.join(['..', 'codes']), 'micropython'))) 

In [ ]:
import config

if config.IS_MICROPYTHON:
    import urequests as requests 
else:
    import requests as requests  

In [ ]:
# schedules = '''
# null_,11:00_,854_|null_,11:00_,7461_|null_,11:02_,19716_|null_,11:04_,13059_|null_,11:05_,852_|
# null_,11:07_,12795_|null_,11:08_,15498_|null_,11:09_,13061_|0_,11:14_,20730_|4_,11:18_,13297_|
# 5_,11:19_,19634_|6_,11:20_,13274_|8_,11:22_,13280_|9_,11:23_,13126_|10_,11:25_,13278_|0_,11:26_,13128_|
# 0_,11:27_,13117_|2_,11:28_,12549_|3_,11:29_,13266_|3_,11:30_,18746_|3_,11:31_,13277_|5_,11:32_,13296_|
# 5_,11:33_,13312_|6_,11:33_,20806_|7_,11:34_,6386_|8_,11:36_,13271_|10_,11:38_,13273_|12_,11:39_,13269_|
# 14_,11:41_,13267_|0_,11:42_,13298_|2_,11:44_,13256_|3_,11:45_,13270_|4_,11:47_,13268_|6_,11:49_,10994_|
# 7_,11:50_,13279_|8_,11:51_,13272_|10_,11:52_,13119_|11_,11:54_,13276_|12_,11:55_,13121_|14_,11:57_,13340_|
# 17_,12:00_,20519_|18_,12:01_,19407_|20_,12:03_,13191_|0_,12:05_,13123_|2_,12:06_,13199_|3_,12:07_,13141_|
# 3_,12:08_,13085_|4_,12:10_,2819_|6_,12:12_,13235_|8_,12:13_,13079_|8_,12:14_,13077_|11_,12:17_,13075_|
# 12_,12:18_,13315_|14_,12:19_,10083_|15_,12:20_,13195_|15_,12:21_,13193_|17_,12:22_,13149_|17_,12:23_,13066_|
# 19_,12:24_,12590_|19_,12:25_,18089_|23_,12:29_,13101_|24_,12:29_,13181_|26_,12:31_,13071_|27_,12:32_,13197_|
# 28_,12:33_,13099_|29_,12:34_,13155_|30_,12:35_,13249_|31_,12:36_,13147_|31_,12:37_,17936_|0_,12:38_,13245_|
# 1_,12:39_,8108_|1_,12:40_,14034_|2_,12:40_,13247_|3_,12:41_,13185_
# '''
# schedules = [schedule for schedule in split_text(schedules)]
# schedules[:3]

In [ ]:
# schedules[-3:]

In [ ]:
def split_text(text):
    text = text.split('@')[0]
    length = len(text)
    start = 0
    end = 0
    
    while end < length - 1 and end >= 0:
        end = text.find('|', start + 1)
        part = text[start + 1:end]
        yield part.split(',')        
        start = end  
        
        
def get_ETA(schedules, bus_stop_code):
    for schedule in schedules:
        if schedule[2] == bus_stop_code + '_': 
            return schedule[:-1]
        
        
def get_minutes_left(schedule):
    minutes_left = schedule[0].replace('_', '')
    if minutes_left.isdigit() and int(minutes_left) >= 0:
        return minutes_left
    return '  '        
    

def query_minutes_left(route_id, route_direction, bus_stop_code):
    
    base_url = 'http://citybus.taichung.gov.tw/ibus/RealRoute/aspx/RealRoute.ashx?Lang=Cht&BusType=0&Data=' + route_id + '_,' + route_direction
    # url_bus_stops = base_url + '&Type=GetStop'  # 站牌名稱 編號 座標
    url_schedules = base_url + '&Type=GetFreshData'  # 站牌編號 到站時刻 
    
    resp = requests.get(url_schedules)
    schedules = resp.text
    schedules = split_text(schedules)
    schedule = get_ETA(schedules, bus_stop_code)
    print(schedule)

    return resp.status_code, resp.reason, get_minutes_left(schedule)

In [ ]:
def query_and_show_minutes_left(route_id, route_direction, bus_stop_code):
    minutes_left = query_minutes_left(route_id, route_direction, bus_stop_code)
    display(minutes_left) 
    
    
def display(minutes_left):
    print(minutes_left if minutes_left else 'NA')    

In [ ]:
import time
import datetime

while True:
    print(datetime.datetime.now())
    query_and_show_minutes_left(route_id = config.route_id, 
                                route_direction = config.route_direction, 
                                bus_stop_code = config.bus_stop_code)
    print('\n')
    
    time.sleep(60 * 1)

In [ ]:
# route_id = config.route_id
# route_direction = config.route_direction 

# base_url = 'http://citybus.taichung.gov.tw/ibus/RealRoute/aspx/RealRoute.ashx?Lang=Cht&BusType=0&Data=' + route_id + '_,' + route_direction
# url_bus_stops = base_url + '&Type=GetStop'  # 站牌名稱 編號 座標
# url_schedules = base_url + '&Type=GetFreshData'  # 站牌編號 到站時刻 

# print(url_bus_stops)
# print(url_schedules)